# Decorators

In [3]:
# There are two different decorators: function decorators and class decorators. More common is the function decorator.

### Function decorators

In [5]:
# Decorator is a function that takes as argument another function and extends the behaviour of this function.
# In other words decorator allows you to add new functionality to an existing function.
#@mydecorator
def dosomething():
    pass

In [6]:
def start_end_decorator(func):
    def wrapper():
        print("Start")
        func()
        print("End")
    return wrapper

def print_name():
    print('Szymon')

print_name()

Szymon


In [7]:
print_name = start_end_decorator(print_name)
print_name()

Start
Szymon
End


In [8]:
# this is the same as above
@start_end_decorator
def print_name():
    print('Szymon')

print_name()

Start
Szymon
End


In [10]:
@start_end_decorator
def add5(x):
    return x+5
add5(10)
# we have to change (add) *args and *kwargs to our decorator function

TypeError: wrapper() takes 0 positional arguments but 1 was given

In [13]:
def start_end_decorator(func):
    def wrapper(*args, **kwargs):
        print("Start")
        func(*args, **kwargs)
        print("End")
    return wrapper

@start_end_decorator
def add5(x):
    return x+5

result = add5(10)
print(result)
# this has printed None, to fix this I have to change decorator function

Start
End
None


In [16]:
def start_end_decorator(func):
    def wrapper(*args, **kwargs):
        print("Start")
        result = func(*args, **kwargs) # here i assigned func() to result
        print("End")
        return result # here I return result
    return wrapper

@start_end_decorator
def add5(x):
    return x+5

total = add5(10)
print(total)

Start
End
15


In [22]:
print(help(add5))
print("----------")
print(add5.__name__)
# help is on function wrapper
# function name is wrapper
# Python got confused about identity of the function, so we have to import functools and use @functools.wraps

Help on function add5 in module __main__:

add5(x)

None
----------
add5


In [24]:
import functools

def start_end_decorator(func):
    @functools.wraps(func) # this will now preserve the information of my used function
    def wrapper(*args, **kwargs):
        print("Start")
        result = func(*args, **kwargs)
        print("End")
        return result
    return wrapper

@start_end_decorator
def add5(x):
    return x+5

print(help(add5))
print("----------")
print(add5.__name__)
# help is on function add5 (not wrapper)
# function name is add5 (not wrapper)

Help on function add5 in module __main__:

add5(x)

None
----------
add5


In [29]:
# Decorators with arguments.
def repeat(number_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            for _ in range(number_times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator_repeat
    
@repeat(number_times=4)
def greet(name):
    print(f'Hello {name}')
    
greet('Szymon')

Hello Szymon
Hello Szymon
Hello Szymon
Hello Szymon


In [35]:
# Nested decorators (we can stack decorators on top of each other).

def start_end_decorator(func):
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("Start")
        result = func(*args, **kwargs)
        print("End")
        return result
    return wrapper

def debug(func):
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        args_repr = [repr(a) for a in args]
        kwargs_repr = [f"{k} = {v!r}" for k,v in kwargs.items()]
        signature = ", ".join(args_repr + kwargs_repr)
        print(f"Calling {func.__name__}({signature})")
        result = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {result!r}")
        return result
    return wrapper

# We apply multiple decorators to a function. They will be executed in the order they are listed.
@debug
@start_end_decorator
def say_hello(name):
    greeting = f'Hello {name}'
    print(greeting)
    return greeting

say_hello('Szymon')
# First of all this will execute debug function and then inside the debug function it will execute the start_end_decorator function
# and then inside the start_end_decorator function it will execute the say_hello function.

Calling say_hello('Szymon')
Start
Hello Szymon
End
'say_hello' returned 'Hello Szymon'


'Hello Szymon'

### Class decorators

In [38]:
class CountCalls:
    def __init__(self, func):
        self.func = func
        self.num_calls = 0
        
    def __call__(self, *args, **kwargs): # this is the same as the inner function in our function decorator
        print('Hi there')

cc = CountCalls(None)
cc()

Hi there


In [43]:
class CountCalls:
    def __init__(self, func):
        self.func = func
        self.num_calls = 0
        
    def __call__(self, *args, **kwargs): # this is the same as the inner function in our function decorator
        self.num_calls += 1
        print(f'This is executed {self.num_calls} times')
        return self.func(*args, **kwargs)

@CountCalls
def say_hello():
    print('Hello')
    
say_hello()
say_hello()
say_hello()
# this short program keeps track of how many times this is executed

This is executed 1 times
Hello
This is executed 2 times
Hello
This is executed 3 times
Hello
